# Construción de algoritmo de prueba
Este código está diseñado para procesar textos y extraer cierta información de una serie de documentos. Una vez que se ha extraído la información deseada, se guarda en un archivo CSV (Comma Separated Values) para su posterior uso o análisis.

Desarrollador: Rosinver Alejandro Vasquez Duran

google colab: https://colab.research.google.com/drive/1xdxcWEzhhAY9tEXqlk9Hzun7uFVXQ_iw?usp=sharing

In [ ]:
!pip install spacy
!python -m spacy download es_core_news_lg

In [104]:
# importar librearias 
import csv
import spacy
import pandas as pd
import numpy as np
import nltk 
import re
from dateutil import parser

In [105]:
# Cargar el modelo de lenguaje de spaCy
nlp = spacy.load("es_core_news_lg")

In [106]:
# como leer un archivo csv delimitado por | y con codificacion utf-8
df = pd.read_csv('documentos.csv', sep='|', encoding='utf-8')
print(df.head())

   document_name                                           document
0             86    Señor/señora: Daniel Roberto Torres Gómez  L...
1            470    Atención: Ana Miguel Jiménez Rodríguez  Le e...
2             16    Oiga!!!! Juan Pablo Fernández Muñoz  Esperam...
3            165    Querido/a Carlos Miguel Pérez Pérez,  Espera...
4            114    Estimadisimo/a Daniel Cristina Rodríguez Gon...


# Explicación:
- El script comienza inicializando una lista vacía llamada "output_data" que se utilizará para almacenar los datos de salida.

- Luego, utiliza dos expresiones regulares para buscar patrones de montos y fechas en los documentos. Estas expresiones regulares se guardan en las variables "monto_pattern" y "fecha_pattern".

- A continuación, el script itera sobre cada fila del dataframe "df" y procesa el documento en cada iteración. Utiliza la librería spacy para realizar un análisis de entidades en el documento y busca entidades de tipo "PER" (persona) y "LOC" (ubicación). Cuando encuentra una entidad de tipo "PER", agrega el nombre de la persona a la lista de salida y cuando encuentra una entidad de tipo "LOC", agrega la ubicación a la lista de salida.

- Luego, utiliza las expresiones regulares para buscar patrones de fechas en el documento y, si encuentra una fecha, la agrega a la lista de salida.

- Finalmente, utiliza la expresión regular para buscar un patrón de monto en el documento y, si encuentra un monto, lo agrega a la lista de salida.

- Una vez que se han procesado todos los documentos, el script crea un dataframe a partir de la lista de salida y lo guarda en un archivo CSV llamado "output.csv".

In [116]:
# Inicializa una lista vacía para almacenar los datos de salida
output_data = []

# Extrae el costo con una expresión regular
monto_pattern = r"\b\d{1,3}(,\d{3})*\.\d{2}\b"
# patron de fecha yyyy- mm-dd o dd-mm-yyyy
fecha_pattern = r"\d{1,4}[/-]\d{1,2}[/-]\d{4}"
fecha_pattern2 =r'\d{4}-\d{2}-\d{2}'

# Itera sobre cada fila del dataframe
for index, row in df.iterrows():
    doc = nlp(row['document'])
    for ent in doc.ents:
        if ent.label_ == 'PER':
            # Agrega el nombre de la persona a la lista de salida
            # agregar 4 tokens del nombre de la persona para filtrar los nombre compuestos por 4 palabras
            output_data.append({'nombre_cliente': ' '.join(ent.text.split()[:4])})
            break
    #buscar localizacion
    for ent in doc.ents:
        if ent.label_ == 'LOC':
            # Agrega la localización a la lista de salida
            output_data[-1]['ciudad'] = ent.text
            break
    #buscar fecha pattern
    matches = re.findall(fecha_pattern , row['document'])
    for match in matches:
        date = parser.parse(match)
        # Agrega la fecha a la lista de salida
        output_data[-1]['fecha'] = date.strftime('%Y-%m-%d')
    #buscar fecha pattern2 
    matches2 = re.findall(fecha_pattern2 , row['document'])
    for match2 in matches2:
        date2 = parser.parse(match2)
        # Agrega la fecha a la lista de salida
        output_data[-1]['fecha'] = date2.strftime('%Y-%m-%d')
        
    monto = re.search(monto_pattern, row['document']).group()
    # cambiar los numeros del formato 000,000,000.00 a 000000000.00
    monto = monto.replace(',', '')
    # Agrega el costo a la lista de salida
    output_data[-1]['monto'] = monto
    
# Crea un dataframe a partir de la lista de salida
output_df = pd.DataFrame(output_data)

# Guarda el dataframe en un archivo csv
output_df.to_csv('output.csv', sep='|', index=False)

print('output.csv >done<')

output.csv >done<


Salida de resultado.csv

In [117]:
#concatenar los archivos csv de documentos y resultado
dfout = pd.concat([df, output_df], axis=1)
#guardar el archivo csv
dfout.to_csv('resultado.csv', sep='|', index=False)

print('resultado.csv >done<')

resultado.csv >done<
